# 2.3 Tagging, using function calling / tool use

(Adapted from https://python.langchain.com/docs/use_cases/tagging) 


## Use case

Tagging means labeling a document with classes such as:

- sentiment
- language
- style (formal, informal etc.)
- covered topics
- political tendency

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/tagging.png?raw=1)

## Overview

Tagging has a few components:

* `function`: Like [extraction](/docs/use_cases/extraction), tagging uses [functions](https://openai.com/blog/function-calling-and-other-api-updates) to specify how the model should tag a document
* `schema`: defines how we want to tag the document

## Setup

### Install dependencies

In [ ]:
%pip install python-dotenv~=1.0 docarray~=0.40.0 pypdf~=5.1 --upgrade --quiet
%pip install langchain~=0.3.7 langchain_openai~=0.2.6 langchain_community~=0.3.5 --upgrade --quiet

# If running locally, you can do this instead:
#%pip install -r ../requirements.txt

### Load environment variables

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# If running in Google Colab, you can use this code instead:
# from google.colab import userdata
# os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
# os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")

### Setup Models

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
api_version = "2024-10-01-preview"
llm = AzureChatOpenAI(deployment_name="gpt-4o", temperature=0.0, api_version=api_version)
embedding_model = AzureOpenAIEmbeddings(model="text-embedding-3-large", api_version=api_version)

## Quickstart

Let's see a very straightforward example of how we can use OpenAI tool calling for tagging in LangChain. We'll use the [`with_structured_output`](/docs/modules/model_io/chat/structured_output) method supported by OpenAI models:

Let's specify a Pydantic model with a few properties and their expected type in our schema.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# LLM

structured_llm = llm.with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | structured_llm

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
tagging_chain.invoke({"input": inp})

If we want JSON output, we can just call `.dict()`

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = tagging_chain.invoke({"input": inp})
res.dict()

As we can see in the examples, it correctly interprets what we want.

The results vary so that we may get, for example, sentiments in different languages ('positive', 'enojado' etc.).

We will see how to control these results in the next section.

## Finer control

Careful schema definition gives us more control over the model's output.

Specifically, we can define:

- possible values for each property
- description to make sure that the model understands the property
- required properties to be returned

Let's redeclare our Pydantic model to control for each of the previously mentioned aspects using enums:

In [ ]:
# Side note - it is also possible to use proper Enums from the standard library
# from enum import Enum
#
# class Sentiment(Enum):
#     happy = "happy"
#     neutral = "neutral"
#     sad = "sad"

class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"]) # Using enum shorthand
    #sentiment: Sentiment = Field(..., description="The sentiment of the text")
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

Classification.model_json_schema()

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

llm = chat_model.with_structured_output(
    Classification
)

chain = tagging_prompt | llm

Now the answers will be restricted in a way we expect!

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.invoke({"input": inp})

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.invoke({"input": inp})

In [ ]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.invoke({"input": inp})

The [LangSmith trace](https://smith.langchain.com/public/38294e04-33d8-4c5a-ae92-c2fe68be8332/r) lets us peek under the hood:

![Image description](https://github.com/langchain-ai/langchain/blob/master/docs/static/img/tagging_trace.png?raw=1)

### Going deeper

* You can use the [metadata tagger](/docs/integrations/document_transformers/openai_metadata_tagger) document transformer to extract metadata from a LangChain `Document`.
* This covers the same basic functionality as the tagging chain, only applied to a LangChain `Document`.